In [8]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from mirisim.config_parser import SimulatorConfig
from mirisim import MiriSimulation
import tso_lrs_sims_setup
from pipetest_tso_lrs_sim import star_scene, lrstso_sim_config

from importlib import reload

In this notebook I'm going to generate simulated MIRI time series LRS (slitless) data, for data management & pipeline testing purposes. To install Mirisim, see the [the public release webpage](http://miri.ster.kuleuven.be/bin/view/Public/MIRISim_Public). The target for the mock observations is WASP-62, an exoplanet host star with the following properties from [the exoplanet encyclopaedia](http://exoplanet.eu/catalog/wasp-62_b/):

* spectral type F7
* T_bb = 6230 K
* K = 8.994

K magnitude of 8.994 corresponds to a flux of 20e3 microJy at 8 micron (value obtained from the ETC), and I'll use this value for the source generation.

Using the ETC, I calculated the following number of groups for a high-SNR but unsaturated image:
ngrps = 100

Three simulations will be executed, for a short and an intermediate-duration exposure. The number of groups will be the same for all 3, as will the number of exposures (nexp = 1). The numbers of integrations will be 10 and 20 (these can be changed very easily).

### Steps in setting up the simulation

This notebook will go through the following steps:

* Create the scene
* Set up the simulation
* Run the simulation

Each step has its own function. Steps 1 and 2 will each write out a .ini file, which will be used as input for the final step.

In [10]:
arr = ['SLITLESSPRISM'] * 2
ngrp = [100] * 2
nints = [10, 20]


## Step 1: Creating the input scene (star_scene)

Here we'll create the input scene for the simulations using the function star_scene(). Arguments:

* scene_file: the filename for the .ini file
* write_cube: write the scene image out to a FITS file (optional; default=False)

The function returns a mirisim.skysim.scenes.CompositeSkyScene object.


In [11]:
scene_ini = star_scene(scene_file='lrstso_star_scene.ini', write_cube=False)

2020-08-06 11:19:56,368 - INFO - Initializing Point
2020-08-06 11:19:56,369 - INFO - Initializing Background


Creating scene. Scene will be written to file lrstso_star_scene.ini
File lrstso_star_scene.ini already exists. Will be overwritten.
lrstso_star_scene.ini exists, overwrite (y/[n])?y


In [12]:
print(scene_ini)

lrstso_star_scene.ini


## Step 2: Configuring the simulation

Now I'll set up the simulations and prepare to run them. Before we write out a new simconfig file, we delete any existing files in the directory. I'll set it up to loop through the different simulations. For this I wrote the function lrstso_sim_config. Check the docstring for descriptions and default values of the arguments. 

The function will write out another .ini file containing the simulation configuration, and it returns the output filename for further use.

In [13]:
old_files = glob.glob('*_simconfig.ini')

for of in old_files:
    os.remove(of)

for (a, g, i) in zip(arr, ngrp, nints):
    sim_ini = lrstso_sim_config(mode='lrs', arr=a, ngrp=g, nint=i, nexp=1, 
                   scene_file=scene_ini, out=True)
    print(sim_ini)

Found scene file lrstso_star_scene.ini
**** Simulation will use the WebbPSF model ****
lrstso_SLITLESSPRISM_100G10I1E_simconfig.ini
Found scene file lrstso_star_scene.ini
**** Simulation will use the WebbPSF model ****
lrstso_SLITLESSPRISM_100G20I1E_simconfig.ini


### Step 3: Run the simulation

In the following step we'll run the simulations for the 2 different cases. For each run, we need 3 input files: the scene, the simulation configuration, and the simulator setup file. The first and last of these remain the same for each run, and we loop through the list of 6 simulation config files.

After the simulation has run, the code renames the output directory to include the simulation settings to the directory.


In [14]:
cfg_files = glob.glob('*_simconfig.ini')

# configure the simulator engine - this requires no editing from the default
simulator_config = SimulatorConfig.from_default()

for f in cfg_files[:1]:
    print(f)
    tmp = f.split('.')
    fcomps = tmp[0].split('_')
    sim = MiriSimulation.from_configfiles(f)
    sim.run()
    outdir = sorted(glob.glob('*_*_mirisim'), key=os.path.getmtime )[-1]
    new_outdir = 'pipetest_lrstso_{0}_{1}_{2}'.format(fcomps[1], fcomps[2], outdir)
    os.rename(outdir, new_outdir)
    print(outdir, new_outdir)


2020-08-06 11:20:25,115 - INFO - Using simulation configuration: lrstso_SLITLESSPRISM_100G10I1E_simconfig.ini
2020-08-06 11:20:25,117 - INFO - Using scene configuration: lrstso_star_scene.ini
2020-08-06 11:20:25,119 - INFO - MIRISim version: 2.3.0b0
2020-08-06 11:20:25,120 - INFO - MIRI Simulation started.
2020-08-06 11:20:25,120 - INFO - Output will be saved to: 20200806_112025_mirisim
2020-08-06 11:20:25,121 - INFO - Storing configs in output directory.


lrstso_SLITLESSPRISM_100G10I1E_simconfig.ini


2020-08-06 11:20:25,557 - INFO - Reading cosmic ray properties from parameter file /Users/kendrew/anaconda3/envs/miricle.devel/lib/python3.7/site-packages/miri/simulators/scasim/cosmic_ray_properties.py
2020-08-06 11:20:25,563 - INFO - Reading detector properties from parameter file /Users/kendrew/anaconda3/envs/miricle.devel/lib/python3.7/site-packages/miri/simulators/scasim/detector_properties.py
2020-08-06 11:20:25,601 - INFO - Storing dither pattern in output directory.
2020-08-06 11:20:25,602 - INFO - Using $CDP_DIR for location of CDP files: /Users/kendrew//CDP
2020-08-06 11:20:25,603 - INFO - Setting up simulated Observation, with following settings:
2020-08-06 11:20:25,603 - INFO - Configuration Path: LRS_SLITLESS
2020-08-06 11:20:25,604 - INFO - Primary optical path: IMA
2020-08-06 11:20:25,605 - INFO - IMA Filter: P750L
2020-08-06 11:20:25,605 - INFO - IMA Subarray: SLITLESSPRISM
2020-08-06 11:20:25,606 - INFO - IMA detector readout mode: FAST
2020-08-06 11:20:25,607 - INFO -

2020-08-06 11:24:47,940 - INFO -   Detector subarray mode is SLITLESSPRISM  [529, 1, 416, 72].
2020-08-06 11:24:47,940 - INFO - Detector temperature = 6.70 K (which affects dark current and read noise).
2020-08-06 11:24:47,942 - INFO - Cosmic ray environment is SOLAR_MIN.
2020-08-06 11:24:47,944 - INFO - Reading cosmic ray library file: '/Users/kendrew/anaconda3/envs/miricle.devel/lib/python3.7/site-packages/miri/simulators/data/cosmic_rays/CRs_SiAs_470_SUNMIN_04.fits'
2020-08-06 11:24:47,985 - INFO - Simulation control flags:
2020-08-06 11:24:47,985 - INFO - 	Quantum efficiency simulation turned OFF.
2020-08-06 11:24:47,986 - INFO - 	Poisson noise simulation turned ON.
2020-08-06 11:24:47,987 - INFO - 	Read noise simulation turned ON.
2020-08-06 11:24:47,987 - INFO - 	Reference pixels simulation turned ON.
2020-08-06 11:24:47,988 - INFO - 	Bad pixels simulation turned ON.
2020-08-06 11:24:47,989 - INFO - 	Dark current simulation turned ON.
2020-08-06 11:24:47,990 - INFO - 	Flat-field 

20200806_112025_mirisim pipetest_lrstso_SLITLESSPRISM_100G10I1E_20200806_112025_mirisim
